#Exercise 5: Conductance-based synaptic currents

In [1]:
#Import the python libraries
import matplotlib.pyplot as plt
plt.ion()
from neuron import h
import numpy
import time
from ipywidgets import widgets, fixed, Layout

In [2]:
%%html
<style type='text/css'>
.widget-inline-hbox .widget-label {
      max-width: 250px;
      min-width: 250px;
}
</style>

In [3]:
#Run NEURON with Python interface
def runneuron(isadaptive=True,amp=1,stimcomp=0,stimx=0.5,tau1=0.5,tau2=20.0,glsoma=0.03,gldend=0.03,gnasoma=120,gksoma=36,Ra=100,cm=1.0):
    my_start_time = time.time()
    h.load_file("stdlib.hoc")                        #A default NEURON library
    h.load_file("stdrun.hoc")                        #A default NEURON library

    h('objref cvode')                                #Define an object cvode
    h('cvode = new CVode()')                         #Make cvode a time step integrator object
    h('cvode.active('+str(isadaptive*1)+')')         #Set the variable time step integration on/off

    h('create soma, dend[3]')                        #Create soma and three dendritic sections

    h.dend[0].connect(h.soma,1,0)                    #Connect the sections to each other
    h.dend[1].connect(h.soma,1,0)
    h.dend[2].connect(h.dend[0],1,0)
    stimtextdict = {0: h.soma, 1: h.dend[0], 2: h.dend[1], 3: h.dend[2]}
    stimnamedict = {0: 'soma', 1: 'dend[0]', 2: 'dend[1]', 3: 'dend[2]'}

    h('access soma')                                 #Make soma the currently accessed section

    for sec in [h.dend[0], h.dend[1], h.dend[2]]:
        sec.insert('pas')                            #Insert passive membrane mechanism to all sections
        sec.nseg = 5                                 #and use 5 sub-compartments for each section
    h.soma.insert('hh')                              #Insert the Hodgkin-Huxley mechanism (includes leak) to soma
    h.soma.nseg = 5                                  

    #Set the passive parameters for each section:
    h.soma.Ra = Ra; h.soma.cm = cm; h.soma.diam = 10; h.soma.L = 20;  h.soma.el_hh = -54.3; h.soma.gl_hh = 0.001*glsoma; h.soma.gnabar_hh = 0.001*gnasoma; h.soma.gkbar_hh = 0.001*gksoma;
    h.dend[0].Ra = Ra; h.dend[0].cm = cm; h.dend[0].diam = 5; h.dend[0].L = 200; h.dend[0].e_pas = -54.3; h.dend[0].g_pas = 0.001*gldend
    h.dend[1].Ra = Ra; h.dend[1].cm = cm; h.dend[1].diam = 2; h.dend[1].L = 100; h.dend[1].e_pas = -54.3; h.dend[1].g_pas = 0.001*gldend
    h.dend[2].Ra = Ra; h.dend[2].cm = cm; h.dend[2].diam = 2; h.dend[2].L = 160; h.dend[2].e_pas = -54.3; h.dend[2].g_pas = 0.001*gldend

    h.dt = 0.025                                     #Set the time step to 0.025 ms
    h.tstop = 300                                    #Continue the simulation until 200 ms

    h('objref syn1, Vrec')                           #Declare objects syn1 and Vrec

    #Make syn1 an Exp2Syn object, stimulating at a given location of a given compartment
    h(stimnamedict[stimcomp]+' syn1 = new Exp2Syn('+str(stimx)+')')
    h.syn1.tau1 = tau1                               #Synaptic stimulation has given rise and
    h.syn1.tau2 = tau2                               #decay time constants
    h.syn1.e = 0                                     #Reversal potential 0 mV (glutamatergic)
    h("""                                                                                                                                                                                                                                      
objref nc, nilstim, fih                              //Declare objects nc and nilstim and fih                                                                                                                                                                      
nc = new NetCon(nilstim, syn1)                       //Make nc a NetCon object that activates syn1 with predefined event times                                                                                                                                                                                      
nc.weight = """+str(0.001*amp)+"""                   //Maximal synaptic conductance amp (default 1 nS (0.001 uS))                                                                                                                                                                                     
fih = new FInitializeHandler("nc.event(100)")        //Make fih an event handler that makes syn1 activate at 100 ms always at initialization                                                                                                                                                                                        
""")
    trec = h.Vector()                                #Save time instants in trec
    trec.record(h._ref_t)
    
    Vrecs = []                                       #Save segment-wise membrane potential values in Vrecs
    recxs = [0.1*x for x in range(0,10)]             #Record at ten locations along each compartment
    for icomp in range(0,4):                         #Iterate across compartments and recording locations
        for ix in range(0,len(recxs)):
            Vrecs.append(h.Vector())                 #Append a vector and record to the added vector
            Vrecs[-1].record(stimtextdict[icomp](recxs[ix])._ref_v, sec=stimtextdict[icomp])            

    h.init()                                         #Initialize state variables
    h.run()                                          #Run the simulations

    plt.close("all")
    f, axs = plt.subplots(2, 4, sharey=True)
    for j in range(0,4):
        axs[0,j].plot(trec,Vrecs[5+len(recxs)*j])
        axs[0,j].set_title(stimnamedict[j])
        axs[0,j].set_xlabel('$t$ (ms)')
        axs[0,j].set_xlim([0, 300])
        axs[0,j].set_ylim([-80, 40])
    axs[0,0].set_ylabel('$V_m(t)$ (mV)')
    axs[1,0].set_ylabel('max $V_m$ (mV)')
    for j in range(0,4):
        axs[1,j].plot(recxs, [max(Vrecs[i+len(recxs)*j]) for i in range(0,len(recxs))])
        axs[1,j].set_xlabel('x')
    plt.suptitle("Simulation run in "+str(time.time() - my_start_time)+" seconds.")
    plt.show()



In [4]:
slider = widgets.interact(runneuron, isadaptive=fixed(True),
                          amp=widgets.FloatSlider(min=0.0,max=30,step=0.1,value=1,description='Amplitude of square pulse current: amp',layout=Layout(width='50%')), 
                          stimcomp=widgets.Dropdown(options={'soma': 0,'dend[0]': 1,'dend[1]': 2,'dend[2]': 3},description='Stimulated compartment: stimcomp',layout=Layout(width='50%')), 
                          stimx=widgets.FloatSlider(min=0.0,max=1.0,step=0.05,value=0.5,description='Location at the compartment: stimx',layout=Layout(width='50%')), 
                          tau1=widgets.FloatSlider(min=0.1,max=15.0,step=0.1,value=0.5,description='Synaptic rise time const: tau1',layout=Layout(width='50%')),
                          tau2=widgets.FloatSlider(min=15.0,max=100.0,step=1,value=20,description='Synaptic decay time const: tau2',layout=Layout(width='50%')),
                          glsoma=widgets.FloatSlider(min=0.0,max=0.5,step=0.01,value=0.03,description='Leak conductance, soma: glsoma',layout=Layout(width='50%')), 
                          gldend=widgets.FloatSlider(min=0.0,max=0.5,step=0.01,value=0.03,description='Leak conductance, dend: gldend',layout=Layout(width='50%')), 
                          gnasoma=widgets.Dropdown(options={'Na+ channels active (120)': 120, 'Na+ channels blocked (0)': 0}, value=120,layout=Layout(width='50%')), 
                          gksoma=widgets.Dropdown(options={'K+ channels active (36)': 36, 'K+ channels blocked (0)': 0}, value=36,layout=Layout(width='50%')), 
                          Ra=widgets.FloatSlider(min=10.0,max=1000.0,step=10,value=100,description='Axial resistance: Ra',layout=Layout(width='50%')),
                          cm=widgets.FloatSlider(min=0.1,max=10.0,step=0.1,value=1.0,description='Membrane capacitance: cm',layout=Layout(width='50%')))


Widget Javascript not detected.  It may not be installed or enabled properly.


#Question 1: The conductance-based synaptic current is described by the following equations:
$$I_{syn}(t) = g_{syn}(t)(E_{syn}-V_m),$$
$$g_{syn}(t) = g_{amp}(\tau_1, \tau_2)(e^{-t/\tau_2} - e^{-t/\tau_1}),$$
where the constant parameter $g_{amp}$ is given a value which makes sure that the maximal value of $g_{syn}(t)$ is constant ("amp" in the sliders above) regardless of the values of $\tau_1$ and $\tau_2$. How do $\tau_1$ and $\tau_2$ affect the shape of the synaptic current? Confirm your theoretical predictions by the above simulations (hint: disable the active Na+ and K+ conductances and set the leak conductances large). The first row of panels shows the membrane potential time course at the center of each compartment, while the second row shows the maximal (over time) membrane potential at each point $x\in[0,1]$ of each of the four compartments.


#Question 2a: Set the HH conductances (gnasoma and gksoma) active. Stimulate the dendritic section furthest away from the soma. How can you make the neuron fire multiple action potentials? <i>Hint: Stimulate at furthest section (dend[2]) with a long decay constant and large Ra, and explore the effect of other parameters.</i>
#***Question 2b***: Set the synaptic current to apply directly to soma. Can you make the neuron fire repetitively? How is the firing different compared to the case in 2b?